# Muffin vs Chihuahua Classification (Custom Dataset)

This section demonstrates binary image classification using a custom muffin vs chihuahua dataset and a CNN architecture.

This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



**Section: Data — Dataset Directory Configuration**

This cell defines the dataset directories used for Muffin vs Chihuahua. Update the paths if your data is located elsewhere.

In [2]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [3]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.

train_1 = r"C:\Users\LENOVO\Downloads\train_split"
test_1 = r"C:\Users\LENOVO\Downloads\test_split"


**Section: Data Preprocessing & Augmentation**

This cell contains the ImageDataGenerator configurations and creates training, validation and test generators.

In [4]:
# OPTIONAL: Split dataset into train/test folders (run once)
import shutil
from pathlib import Path
import random

def split_dataset(source_dir, train_ratio=0.8):
    """
    Split dataset into train and test directories
    source_dir: Path to folder containing class subfolders (chihuahua, muffin)
    train_ratio: Percentage of data for training (0.8 = 80%)
    """
    source = Path(source_dir)
    parent = source.parent
    
    # Create train and validation directories (avoid naming conflict)
    train_dir = parent / "train_split"
    test_dir = parent / "test_split"
    
    # Get all class folders (chihuahua, muffin)
    for class_folder in source.iterdir():
        if class_folder.is_dir():
            class_name = class_folder.name
            
            # Create class folders in train and test
            (train_dir / class_name).mkdir(parents=True, exist_ok=True)
            (test_dir / class_name).mkdir(parents=True, exist_ok=True)
            
            # Get all images in this class
            images = list(class_folder.glob("*.*"))
            random.shuffle(images)
            
            # Split images
            split_idx = int(len(images) * train_ratio)
            train_images = images[:split_idx]
            test_images = images[split_idx:]
            
            # Copy images to train folder
            for img in train_images:
                shutil.copy2(img, train_dir / class_name / img.name)
            
            # Copy images to test folder
            for img in test_images:
                shutil.copy2(img, test_dir / class_name / img.name)
            
            print(f"{class_name}: {len(train_images)} train, {len(test_images)} test")
    
    print(f"\nDataset split complete!")
    print(f"Train directory: {train_dir}")
    print(f"Test directory: {test_dir}")

# Remove the hash and run once to split the dataset
# split_dataset(r"C:\Users\LENOVO\Downloads\test", train_ratio=0.8)

In [5]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [6]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_1,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_1,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_1,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 758 images belonging to 3 classes.
Found 189 images belonging to 3 classes.
Found 189 images belonging to 3 classes.
Found 237 images belonging to 3 classes.
Found 237 images belonging to 3 classes.


In [7]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

from tensorflow.keras import regularizers



initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)



model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.5277 - loss: -10003.2852 - val_accuracy: 0.5397 - val_loss: -63448.3008
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.5277 - loss: -10003.2852 - val_accuracy: 0.5397 - val_loss: -63448.3008
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5409 - loss: -833711.8750 - val_accuracy: 0.5397 - val_loss: -2888982.2500
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5409 - loss: -833711.8750 - val_accuracy: 0.5397 - val_loss: -2888982.2500
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.5409 - loss: -12075809.0000 - val_accuracy: 0.5397 - val_loss: -34206580.0000
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.5409 - loss: -12075809.0000 - val_accuracy: 0.5397 - val_loss: -34206580.0000
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.5409 - loss: -97416656.0000 - val_accuracy: 0.5397 - val_loss: -212706944.0000
Epoch 5/10
24/24 ━━━━━━

In [10]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 972ms/step - accuracy: 0.5401 - loss: -64797044736.0000
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 972ms/step - accuracy: 0.5401 - loss: -64797044736.0000
Test Accuracy: 0.5400843620300293
Test Accuracy: 0.5400843620300293


In [11]:
# SAVE THE IMPROVED MODEL
model.save('exercise_6_trained_model_improved.h5')

In [12]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [13]:
# Example usage - Run 1 and 2 prediction and confidence:
predict_image(r"C:\Users\LENOVO\Downloads\test_split\chihuahua\img_0_1082.jpg", model_path='exercise_6_trained_model_improved.h5')
predict_image(r"C:\Users\LENOVO\Downloads\test_split\muffin\img_0_423.jpg", model_path='exercise_6_trained_model_improved.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
Prediction: Chihuahua (confidence: 1.00)
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Prediction: Chihuahua (confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Prediction: Chihuahua (confidence: 1.00)


# Cats vs Dogs Classification (Kaggle)

This section demonstrates how to use the Kaggle Cats vs Dogs dataset with the improved CNN architecture.

In [14]:
# DATASET DIRECTORY CONFIGURATION (Cats vs Dogs)
catsdogs_train_dir = r"C:\Users\LENOVO\Downloads\cats_and_dogs\training_set\training_set"
catsdogs_test_dir = r"C:\Users\LENOVO\Downloads\cats_and_dogs\test_set\test_set"

**Section: Model Architecture (Custom CNN)**

This cell defines the improved custom CNN architecture with L2 regularization and dropout.

In [15]:
# DATA PREPROCESSING & AUGMENTATION (Cats vs Dogs)
catsdogs_img_size = (128, 128)
catsdogs_batch_size = 32
catsdogs_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
catsdogs_test_datagen = ImageDataGenerator(rescale=1./255)

catsdogs_train_generator = catsdogs_train_datagen.flow_from_directory(
    catsdogs_train_dir,
    target_size=catsdogs_img_size,
    batch_size=catsdogs_batch_size,
    class_mode='binary',
    subset='training'
)
catsdogs_val_generator = catsdogs_train_datagen.flow_from_directory(
    catsdogs_train_dir,
    target_size=catsdogs_img_size,
    batch_size=catsdogs_batch_size,
    class_mode='binary',
    subset='validation'
)
catsdogs_test_generator = catsdogs_test_datagen.flow_from_directory(
    catsdogs_test_dir,
    target_size=catsdogs_img_size,
    batch_size=catsdogs_batch_size,
    class_mode='binary',
    shuffle=False
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


**Section: Model Compilation**

This cell compiles the model with optimizer, loss and metrics.

In [16]:
# IMPROVED CNN MODEL FOR CATS VS DOGS

from tensorflow.keras import regularizers

initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


catsdogs_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(catsdogs_img_size[0], catsdogs_img_size[1], 3),
                kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


catsdogs_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

**Section: Training**

This cell runs the model training process.

In [ ]:
# TRAINING THE CNN (Cats vs Dogs)
catsdogs_history = catsdogs_model.fit(
    catsdogs_train_generator,
    epochs=10,
    validation_data=catsdogs_val_generator
)

# EVALUATE THE MODEL (Cats vs Dogs)
catsdogs_test_loss, catsdogs_test_acc = catsdogs_model.evaluate(catsdogs_test_generator)
print(f"Cats vs Dogs Test Accuracy: {catsdogs_test_acc}")

# SAVE THE IMPROVED MODEL (Cats vs Dogs)
catsdogs_model.save('exercise_6_custom_senibalo.h5')

# SIMPLE INFERENCE SCRIPT (Cats vs Dogs)
def catsdogs_predict_image(img_path, model_path='exercise_6_custom_senibalo.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=catsdogs_img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Dog" if pred >= 0.5 else "Cat"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 332s 2s/step - accuracy: 0.4983 - loss: 0.8998 - val_accuracy: 0.5003 - val_loss: 0.7864
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 332s 2s/step - accuracy: 0.4983 - loss: 0.8998 - val_accuracy: 0.5003 - val_loss: 0.7864
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 170s 845ms/step - accuracy: 0.5139 - loss: 0.7565 - val_accuracy: 0.4984 - val_loss: 0.7394
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 170s 845ms/step - accuracy: 0.5139 - loss: 0.7565 - val_accuracy: 0.4984 - val_loss: 0.7394
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 151s 749ms/step - accuracy: 0.5166 - loss: 0.7260 - val_accuracy: 0.5690 - val_loss: 0.7096
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 151s 749ms/step - accuracy: 0.5166 - loss: 0.7260 - val_accuracy: 0.5690 - val_loss: 0.7096
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 148s 736ms/step - accuracy: 0.5384 - loss: 0.7108 - val_accuracy: 0.5403 - val_loss: 0.7093
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 148s 736ms/step - accuracy: 0.5384 - loss:

Cats vs Dogs Test Accuracy: 0.682649552822113


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Prediction: Cat (confidence: 0.33)
Prediction: Cat (confidence: 0.33)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Prediction: Dog (confidence: 0.61)
Prediction: Dog (confidence: 0.61)


In [18]:
# Example usage
catsdogs_predict_image(r"C:\Users\LENOVO\Downloads\cats_and_dogs\test_set\test_set\cats\cat.4975.jpg")
catsdogs_predict_image(r"C:\Users\LENOVO\Downloads\cats_and_dogs\test_set\test_set\dogs\dog.4942.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
Prediction: Cat (confidence: 0.33)
Prediction: Cat (confidence: 0.33)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Prediction: Dog (confidence: 0.61)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Prediction: Dog (confidence: 0.61)


**Section: Evaluation**

This cell evaluates the trained model on the test set and prints accuracy (answer location for a).

# Muffin vs Chihuahua — ResNet CNN Architecture

This section uses ResNet50 (transfer learning) on the Muffin vs Chihuahua dataset. It performs preprocessing suited for ResNet, trains a transfer-learning model, evaluates it, and saves the trained model.

**Section: ResNet (Model, Training, Eval, Save, Inference)**

This section contains the ResNet50 transfer-learning workflow for Muffin vs Chihuahua.

In [19]:
# RESNET50: imports & parameters
from tensorflow.keras import regularizers

# ResNet expects 224x224 images
resnet_img_size = (224, 224)
resnet_batch_size = 32

# Use a smaller batch for limited GPU/CPU memory if needed
# resnet_batch_size = 16

In [20]:
# RESNET50: data generators (preprocessing suited for ResNet)
resnet_train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input,
    validation_split=0.2
)
resnet_test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input)

resnet_train_generator = resnet_train_datagen.flow_from_directory(
    train_1,
    target_size=resnet_img_size,
    batch_size=resnet_batch_size,
    class_mode='binary',
    subset='training'
)
resnet_val_generator = resnet_train_datagen.flow_from_directory(
    train_1,
    target_size=resnet_img_size,
    batch_size=resnet_batch_size,
    class_mode='binary',
    subset='validation'
)
resnet_test_generator = resnet_test_datagen.flow_from_directory(
    test_1,
    target_size=resnet_img_size,
    batch_size=resnet_batch_size,
    class_mode='binary',
    shuffle=False
)

Found 758 images belonging to 3 classes.
Found 189 images belonging to 3 classes.
Found 189 images belonging to 3 classes.
Found 237 images belonging to 3 classes.
Found 237 images belonging to 3 classes.


In [21]:
# RESNET50: build model (feature extraction)
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(resnet_img_size[0], resnet_img_size[1], 3)
)
base_model.trainable = False  # freeze base for feature extraction

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(1, activation='sigmoid')(x)

resnet_model = models.Model(inputs=base_model.input, outputs=preds)
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 58s 1us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 58s 1us/step


In [22]:
# RESNET50: train (few epochs for demo; increase for better performance)
resnet_history = resnet_model.fit(
    resnet_train_generator,
    epochs=5,
    validation_data=resnet_val_generator
)

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 128s 5s/step - accuracy: 0.5172 - loss: -2.7920 - val_accuracy: 0.5397 - val_loss: -6.8877
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 128s 5s/step - accuracy: 0.5172 - loss: -2.7920 - val_accuracy: 0.5397 - val_loss: -6.8877
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.5409 - loss: -10.8777 - val_accuracy: 0.5397 - val_loss: -15.7648
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.5409 - loss: -10.8777 - val_accuracy: 0.5397 - val_loss: -15.7648
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 87s 4s/step - accuracy: 0.5409 - loss: -19.4120 - val_accuracy: 0.5397 - val_loss: -25.2013
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 87s 4s/step - accuracy: 0.5409 - loss: -19.4120 - val_accuracy: 0.5397 - val_loss: -25.2013
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.5409 - loss: -28.7135 - val_accuracy: 0.5397 - val_loss: -35.1357
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.5409 - loss: -28.7135 - val_accuracy

In [23]:
# RESNET50: evaluate, save, and example predictions
# Evaluate and mark the answer location for
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(resnet_test_generator)
print(f"ResNet Test Accuracy: {resnet_test_acc}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.5401 - loss: -43.8318
ResNet Test Accuracy: 0.5400843620300293
8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.5401 - loss: -43.8318
ResNet Test Accuracy: 0.5400843620300293


In [24]:
# Save the ResNet model and mark location for
resnet_model.save('exercise_6_trained_resnet50.h5')

In [ ]:
# RESNET: single-image prediction helper (uses ResNet preprocessing and correct input size)
from tensorflow.keras.preprocessing import image

def predict_image_resnet(img_path, model_path='exercise_6_trained_resnet50.h5'):
    """Load an image, resize to ResNet input size, apply preprocess_input, and predict."""
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=resnet_img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    # Use ResNet's preprocess_input (handles scaling/normalization expected by the pretrained model)
    img_array = tf.keras.applications.resnet.preprocess_input(img_array)
    pred = model.predict(img_array)[0,0]
    # For this binary setup: label mapping (consistent with training) -- adjust threshold if needed
    label = 'Muffin' if pred >= 0.5 else 'Chihuahua'
    print(f"Prediction: {label} (raw score: {pred:.4f})")

# Predictions for Run 1 and Run 2 using the ResNet model
predict_image_resnet(r"C:\Users\LENOVO\Downloads\test_split\chihuahua\img_1_153.jpg", model_path='exercise_6_trained_resnet50.h5')
predict_image_resnet(r"C:\Users\LENOVO\Downloads\test_split\muffin\img_0_423.jpg", model_path='exercise_6_trained_resnet50.h5')


ValueError: Input 0 of layer "functional_80" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(1, 128, 128, 3)

**Section: Cats vs Dogs (Dataset header)**

This section contains the Cats vs Dogs workflow using the same improved CNN architecture for comparison.